In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

def find_min_datetime(series):
    # Ensure the series is of datetime type
    series = pd.to_datetime(series)
    
    # Find and return the minimum datetime value
    return series.min()

def find_max_datetime(series):
    # Ensure the series is of datetime type
    series = pd.to_datetime(series)
    
    # Find and return the maximum datetime value
    return series.max()

def get_datetime_range(series):
    # Ensure the series is of datetime type
    series = pd.to_datetime(series)
    
    # Find and return the range of datetime values
    return (series.min(), series.max())

def create_time_range(start, end):
    # Create a range of datetime values
    time_range = {
        "start": datetime.strptime(start, "%Y-%m-%d %H:%M:%S"),
        "end": datetime.strptime(end, "%Y-%m-%d %H:%M:%S")
    }
    
    return time_range

def randomDate(time_range):
    while True:
        try:
            start = time_range["start"]
            end = time_range["end"]
            delta = end - start
            int_delta = delta.days
            random_day = random.randint(0, int_delta)
            result = (start + timedelta(days=random_day)).date().strftime("%Y-%m-%d")
            datetime.strptime(result, "%Y-%m-%d")  # Validate the date
            if result.endswith("02-29"):
                continue  # Avoid 29th February
            return result
        except ValueError:
            continue

def createADateLaterThan(date):
    while True:
        try:
            result = (date + timedelta(days=random.randint(1, 365))).date().strftime("%Y-%m-%d")
            datetime.strptime(result, "%Y-%m-%d")  # Validate the date
            if result.endswith("02-29"):
                continue  # Avoid 29th February
            return result
        except ValueError:
            continue
        
def isDateBetween(date, start, end):
    return datetime.strptime(start,"%Y-%m-%d") <= datetime.strptime(date,"%Y-%m-%d") \
                                            <= datetime.strptime(end,"%Y-%m-%d") 

In [2]:
CHONMON = pd.read_csv('ChonMon.csv')
PHIEUDATMON = pd.read_csv('PhieuDatMon.csv')
NHANVIEN = pd.read_csv('NhanVien.csv')
CHINHANH = pd.read_csv('ChiNhanh.csv')
DOICN = pd.read_csv('DoiCN.csv')

In [28]:
def findValidBranch(NgayLap, MaNV):
    history = DOICN[DOICN['MaNV'] == MaNV]
    NhanVien = NHANVIEN[NHANVIEN['MaNV'] == MaNV]
    for index, row in history.iterrows():
        if isDateBetween(NgayLap, row['NgayBatDau'], row['NgayKetThuc']):
            return row['MaCN']
    return NhanVien['CN_HienTai'].values[0]

In [29]:
for index, row in PHIEUDATMON.iterrows():
    NgayLap = datetime.strptime(row['NgayLap'], "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d")
    MaNV = row['MaNV']
    PHIEUDATMON.at[index, 'MaCN'] = findValidBranch(NgayLap, MaNV)

# Save the updated DataFrame to a CSV file
PHIEUDATMON.to_csv('PhieuDatMon.csv', index=False)

In [3]:
PHIEUDATMON = pd.read_csv('PhieuDatMon.csv')
PHIEUDATMON['MaCN'] = PHIEUDATMON['MaCN'].astype(int)
BAN = pd.read_csv('Ban.csv')
BAN['MaCN'] = BAN['MaCN'].astype(int)

In [4]:
MaBan = []
for index, row in PHIEUDATMON.iterrows():
    MaCN = row['MaCN']
    BAN_CN = BAN[BAN['MaCN'] == MaCN]
    MaBan.append(random.choice(BAN_CN['MaBan'].values))


In [13]:
PHIEUDATMON['MaBan'] = MaBan
PHIEUDATMON = PHIEUDATMON.drop(columns=['sql_command'], inplace=True)
PHIEUDATMON.to_csv('PhieuDatMon.csv', index=False)

KeyError: "['sql_command'] not found in axis"

In [12]:
# Generate insert SQL commands
def create_insert_command(index_row):
    index, row = index_row
    values = "', '".join([str(value) for value in row.values])
    return f"INSERT INTO PHIEUDATMON ({', '.join(PHIEUDATMON.columns)}) VALUES ('{values}');"

insert_commands = list(map(create_insert_command, PHIEUDATMON.iterrows()))

# Write the commands to an SQL file
with open('../src/backend/database/Insert_PhieuDatMon.sql', 'w') as file:
    for command in insert_commands:
        file.write(command + '\n')